In [24]:
# 필수 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
from sklearn.preprocessing import StandardScaler

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
cp -r /content/drive/MyDrive/Colab\ Notebooks/final_project/data /content

In [7]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (8,854 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126308 files and dire

In [27]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [28]:
# pyarrow 설치 (parquet 파일 읽기용)
!pip install pyarrow

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

In [29]:
import os

# 현재 디렉토리의 모든 파일 확인
print(os.listdir())

['.config', '201809_test_청구정보.parquet', 'sample_data', '201811_test_청구정보.parquet', '201812_test_청구정보.parquet', '201807_test_회원정보.parquet', 'drive', '201808_test_청구정보.parquet', '201810_test_청구정보.parquet', 'data', '201807_test_청구정보.parquet']


In [30]:
import pyarrow.parquet as pq

# 의심되는 파일이 실제 parquet인지 확인
pq.ParquetFile('201807_test_청구정보.parquet')

In [31]:
# 데이터 로드: 6개월 청구정보 parquet 파일 병합
file_list = [
    '201807_test_청구정보.parquet',
    '201808_test_청구정보.parquet',
    '201809_test_청구정보.parquet',
    '201810_test_청구정보.parquet',
    '201811_test_청구정보.parquet',
    '201812_test_청구정보.parquet'
]

df_all = pd.concat([pd.read_parquet(f) for f in file_list], ignore_index=True)
print("병합 완료: ", df_all.shape)

병합 완료:  (600000, 46)


In [32]:
# 결측치 처리
missing_cols = df_all.columns[df_all.isnull().any()]
missing_cols

Index([], dtype='object')

In [33]:
# 불필요한 컬럼 제거
cols_to_drop = ['대표결제방법코드', '청구서발송여부_R3M', '청구금액_R3M', '포인트_마일리지_환산_B0M']
df_all.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print("컬럼 제거 완료")

컬럼 제거 완료


In [20]:
# # Segment 컬럼 가져오기
# # parquet 파일 열기
# df01 = pd.read_parquet("/content/data/test/1.회원정보/201807_test_회원정보.parquet")
#     # "/content/data/test/1.회원정보/201807_test_회원정보.parquet")/content/201807_test_회원정보.parquet
# df01

In [21]:
# df01['Segment']

In [34]:
# '대표청구지고객주소구분코드' 컬럼 제거
df_all.drop(columns=['대표청구지고객주소구분코드'], inplace=True, errors='ignore')

In [35]:
# 데이터 개요 확인
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 41 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   기준년월              600000 non-null  int64 
 1   ID                600000 non-null  object
 2   대표결제일             600000 non-null  int64 
 3   대표청구서수령지구분코드      600000 non-null  object
 4   청구서수령방법           600000 non-null  object
 5   청구서발송여부_B0        600000 non-null  int64 
 6   청구서발송여부_R6M       600000 non-null  int64 
 7   청구금액_B0           600000 non-null  int64 
 8   청구금액_R6M          600000 non-null  int64 
 9   포인트_마일리지_건별_B0M   600000 non-null  int64 
 10  포인트_마일리지_건별_R3M   600000 non-null  int64 
 11  포인트_포인트_건별_B0M    600000 non-null  int64 
 12  포인트_포인트_건별_R3M    600000 non-null  int64 
 13  포인트_마일리지_월적립_B0M  600000 non-null  int64 
 14  포인트_마일리지_월적립_R3M  600000 non-null  int64 
 15  포인트_포인트_월적립_B0M   600000 non-null  int64 
 16  포인트_포인트_월적립_R3M   600000 non-null  int

In [36]:
# Parquet로 저장
df_all.to_parquet("df_all.parquet", index=False)

# 파일 다운로드
from google.colab import files
files.download("df_all.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # CSV 파일 불러오기
# df = pd.read_csv('df_all_test.csv', encoding='cp949') # 경로는 필요 시 수정

# # Parquet 파일로 저장
# df.to_parquet('df_all_test.parquet', index=False)

In [ ]:
# ls

In [ ]:
# # 한글이 깨지지 않도록 utf-8-sig 인코딩 적용
# df_all.to_csv('df_all_final.csv', index=False, encoding='utf-8-sig')

# from google.colab import files
# files.download('df_all_final.csv')

In [ ]:
# # 이상치 확인
# def detect_outliers_iqr(df, column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
#     print(f"[{column}] 이상치 개수: {len(outliers)}")
#     return outliers[['ID', column]]

# # 분석 대상 컬럼 리스트
# target_columns = ['할인금액_R3M', '혜택수혜금액']

# # 이상치 탐지 결과 저장
# outlier_results = {}

# # 각 컬럼별 이상치 확인
# for col in target_columns:
#     outliers = detect_outliers_iqr(df_all, col)
#     outlier_results[col] = outliers

In [ ]:
# # 데이터 샘플 추출
# # 1. (선택) 데이터 샘플링: df_all에서 500개 샘플 추출
# df_all_sample = df_all.sample(n=500, random_state=42)  # 또는 head(n)

# # 2. CSV 파일로 저장 (인코딩은 Excel 호환 위해 cp949 사용)
# df_all_sample.to_csv('df_all_sample.csv', index=False, encoding='cp949')

# # 3. 로컬로 다운로드
# from google.colab import files
# files.download('df_all_sample.csv')

In [ ]:
# # 1. 컬럼명 확인
# print(df_all_sample.columns.tolist())

# # 2. 데이터 미리보기
# print(df_all_sample.head(3))

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant

# def calculate_vif(X):
#     X = add_constant(X)  # 상수항 추가 (intercept)
#     vif_data = pd.DataFrame()
#     vif_data["feature"] = X.columns
#     vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
#     return vif_data

In [ ]:
# # 수치형 데이터만 다중공선성(VIF)계산
# numeric_cols = df_all.select_dtypes(include='number')
# vif_result = calculate_vif(numeric_cols)
# print(vif_result)

In [ ]:
# # parquet 파일 열기
# df1 = pd.read_parquet("201807_test_청구정보.parquet")

# # csv로 저장
# df1.to_csv("201807_test_청구정보.csv", index=False)


In [ ]:
# # parquet 파일 열고 csv로 저장
# df2 = pd.read_parquet("201807_test_청구정보.parquet")
# df2.to_csv("201807_test_청구정보.csv", index=False)

# df3 = pd.read_parquet("201807_test_청구정보.parquet")
# df3.to_csv("201807_test_청구정보.csv", index=False)

# df4 = pd.read_parquet("201807_test_청구정보.parquet")
# df4.to_csv("201807_test_청구정보.csv", index=False)

# df5 = pd.read_parquet("201807_test_청구정보.parquet")
# df5.to_csv("201807_test_청구정보.csv", index=False)

# df6 = pd.read_parquet("201807_test_청구정보.parquet")
# df6.to_csv("201807_test_청구정보.csv", index=False)